<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/agents_and_routing/examples_agents_with_critique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ollama langchain_community --quiet

host="localhost:11434"
modelid="gemma3:12b" # "llama3.2"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")


In [2]:
!pip install openai --quiet

In [3]:
get_ipython().system_raw(f"ollama pull {modelid}")

In [4]:
from openai import OpenAI


llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )




In [5]:
def call(system_prompt: str, message: str) -> str:
  completion = llm.chat.completions.create(
      model=modelid,
      messages=[
          {"role": "system", "content": system_prompt},
          {
              "role": "user",
              "content": message,
          },
      ],
  )

  return completion.choices[0].message.content

In [6]:
def marketing_campaign_agent(product : str):
  """

     Your task is to create an compelling marketing campaign plan for the product.

    ## Product Description
    {product}

     ## Marketing Campaign Plan:L
     Your marketing campaign plan:

  """
  return call ("your are a marketing campaign export.", marketing_campaign_agent.__doc__.format(product=product) )

In [7]:
def ready_validation_agent(marketing_text : str) -> bool:
  """

  Attached is a product descipription of a new product we are launching.

  Also attached is a draft marketing campaign plan for the product launch.

     Your task is to answer with
     - TRUE if this is the final version of the marketing text and ready to publish.
     - FALSE if this is not the final version of the marketing text and we need to improve it.

     Only answer with TRUE or FALSE. add nothing else to your response.

  ## Product Description
  {product}

  ## Marketing Text
  {text}


    """
  result = call ("your are a marketing campaign export.", ready_validation_agent.__doc__.format(product=product, text=marketing_text) )
  print (result)
  return result.strip().lower() == "true"

In [8]:
def refinements_agent(product : str, marketing_text : str) -> str:
  """

  Attached is a product descipription of a new product we are launching.

  Also attached is a draft marketing campaign plan for the product launch.

  Provide a critique of this draft marketing plan in preparation of the final version.

  ## Product Description
  {product}

  ## Marketing Text
  {text}

  ## Critique
  Your critique:

  """
  return call ("your are a marketing campaign export.", refinements_agent.__doc__.format(product=product, text=marketing_text) )

In [11]:
def refine_agent(product : str, marketing_text : str, refinements : str ) -> str:
  """

  Attached are 3 documents
    - a product descipription of a new product we are launching.
    - a draft marketing campaign plan for the product launch.
    - a critique of this draft marketing plan in preparation of the final version

  Please write a final version of the marketing campaign plan taking into account the feedback in the critique.

  ## Product Description
  {product}

  ## Marketing Text
  {text}

  ## Critique
  {refinements}

  ## Final Marketing Text
  Your final version of the marketing campaign plan:


  """
  return call ("your are a marketing campaign export.", refinements_agent.__doc__.format(product=product, text=marketing_text, refinements=refinements) )

In [10]:
product = "iPhone 15"

iteration=1
marketing_text = marketing_campaign_agent(product)
while not ready_validation_agent(marketing_text) and iteration<1:
  print (f"ITERATION: {iteration}" )
  refinements = refinements_agent(product, marketing_text)
  next_text_iteration = refine_agent(product, marketing_text, refinements)
  print (refinements)
  iteration+=1
  marketing_text = next_text_iteration


KeyError: 'product'

In [ ]:
print (next_text_iteration)


In [ ]:
print (marketing_text)
print ("###### critique")
print (is_finished)
print (refinements)
print ("###### next text iteration")
print (next_text_iteration)